In [1]:
import pandas as pd
import os
filenames = [_.split('.')[0] for _ in os.listdir('./') if '.csv' in _]
print(filenames)

dfs = [pd.read_csv(filename + '.csv') for filename in filenames]
df_orders, df_prod, df_order_items, df_customers = dfs
print([len(_) for _ in [df_orders, df_prod, df_order_items, df_customers]])

['noahs-orders', 'noahs-products', 'noahs-orders_items', 'noahs-customers']
[213232, 1278, 426541, 8260]


In [2]:
bake = df_order_items[ df_order_items['sku'].str.startswith('BK') ]
bakery = df_order_items[ df_order_items['sku'].str.startswith('BKY') ]
print( 'assert/ BKY is bakery:', len(bake) == len(bakery) )
bake.head()

assert/ BKY is bakery: True


,orderid,sku,qty,unit_price
18,1009,BKY1679,2,1.08
294,1144,BKY5758,1,1.34
316,1157,BKY4760,1,1.48
360,1179,BKY2596,1,1.37
591,1299,BKY6734,1,1.07


In [3]:
bakery_orders = pd.merge(bakery, df_orders, on='orderid', sort=True)
print( len(bakery_orders), '/len orders')
bakery_orders.head()

6648 /len orders


,orderid,sku,qty,unit_price,customerid,ordered,shipped,items,total
0,1009,BKY1679,2,1.08,5294,2017-01-31 08:18:29,2017-01-31 08:18:29,NaN,23.63
1,1144,BKY5758,1,1.34,2881,2017-02-01 15:33:28,2017-02-01 15:33:28,NaN,1.34
2,1157,BKY4760,1,1.48,4717,2017-02-01 16:50:20,2017-02-01 16:50:20,NaN,5.34
3,1179,BKY2596,1,1.37,2446,2017-02-01 20:25:07,2017-02-07 14:00:00,NaN,4.21
4,1299,BKY6734,1,1.07,3930,2017-02-03 00:59:19,2017-02-03 14:00:00,NaN,5.53


In [4]:
def find_2017_5am_bakery (dt_string):
    date, time = dt_string.split()
    return 0<int(time.split(':')[0])<6
early_morning_bakery_orders = bakery_orders[ bakery_orders['ordered'].apply(lambda x: find_2017_5am_bakery (x)) ]
print( len(early_morning_bakery_orders), '/len orders')
early_morning_bakery_orders.head()

187 /len orders


,orderid,sku,qty,unit_price,customerid,ordered,shipped,items,total
38,2794,BKY5054,1,1.19,3903,2017-02-17 03:33:43,2017-02-17 04:15:00,NaN,8.81
39,2794,BKY7716,1,1.47,3903,2017-02-17 03:33:43,2017-02-17 04:15:00,NaN,8.81
53,3328,BKY4760,1,1.42,1611,2017-02-22 05:22:22,2017-02-22 18:00:00,NaN,18.19
91,4524,BKY4760,1,1.46,6202,2017-03-06 05:41:34,2017-03-06 19:15:00,NaN,4.89
103,5117,BKY1679,1,1.46,3301,2017-03-12 05:25:50,2017-03-12 16:45:00,NaN,3.65


In [5]:
res = pd.merge(early_morning_bakery_orders, df_customers, on='customerid', sort=True)
print( len(res), '/len res')
res.groupby(['name','phone']).size().sort_values(ascending=False).head(4)

187 /len res


name            phone       
Renee Harmon    607-231-3605    6
Robin Davis     315-629-9164    4
Lindsey Mays    631-280-4337    3
Scott Hayes II  631-923-6868    3
dtype: int64